<div class="alert alert-block alert-success">
    
# FIT5196 Assessment 2
#### Student Name: Tsz Yan CHUNG
#### Student ID: 32973381

Date: 25/09/2022


Environment: Google colab

</div>

-------------------------------------

<div class="alert alert-block alert-danger">
    
## Table of Contents

</div>    

[1. Introduction](#Intro) <br>
[2. Importing Libraries](#libs) <br>
[3. Examining Review Files](#read) <br>
[4. Fixing dirty.csv](#dirty) <br>
$\;\;\;\;$[4.1. Inspect dirty.csv](#i_dirty) <br>
$\;\;\;\;$[4.2. Fixing date column](#date) <br>
$\;\;\;\;$[4.3. Fixing order_type column](#o_type) <br>
$\;\;\;\;$[4.4. Fixing branch code](#br_code) <br>
$\;\;\;\;$[4.5. Fixing order item](#o_item) <br>
$\;\;\;\;\;\;\;$[4.5.1. Fixing Breakfast item](#breakfast_item) <br>
$\;\;\;\;\;\;\;$[4.5.2. Fixing Lunch item](#lunch_item) <br>
$\;\;\;\;\;\;\;$[4.5.3. Fixing Dinner item](#dinner_item) <br>
$\;\;\;\;$[4.6. Fixing order price](#o_price) <br>
$\;\;\;\;$[4.7. Fixing customer latitude and longitude value](#latlon) <br>
$\;\;\;\;$[4.8. Fixing distance](#dist) <br>
$\;\;\;\;$[4.9. Fixing loyalty](#loyalty) <br>
[5. Imputation for missing.csv](#missing) <br>
$\;\;\;\;$[5.1. Imputing missing branch codes](#br_missing) <br>
$\;\;\;\;$[5.2. Imputing missing delivery fee ](#fee_missing) <br>
[6.  Detect and remove Outliers](#outliers) <br>
[7. Exporting CSV Files](#write) <br>
[8. Summary](#summary) <br>
[9. References](#ref)


-------------------------------------

<div class="alert alert-block alert-warning">

## 1.  Introduction  <a class="anchor" name="Intro"></a>
    
</div>

Three tasks will be performed in this assessment:

1. Detect and fix errors in dirty_data.csv
2. Detect and remove outlier rows in outliers_data.csv
3. Impute missing values in missin_data .csv

These tasks will be performed separately, while some of them might provide insight in other tasks, while each csv file contains 500 records of data.

This notebook aims to demonstrate the methodology to achieve the correct result.

-------------------------------------

<div class="alert alert-block alert-warning">
    
## 2. Importing Libraries  <a class="anchor" name="libs"></a>
 </div>

In [ ]:
# Import libraries
import pandas as pd
import os
import re
import numpy as np
import random
from datetime import datetime as dt
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import math


In [ ]:
# This task was conducted on Google colab
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


-------------------------------------

<div class="alert alert-block alert-warning">
    
## 3. Set up directory and read files <a class="anchor" name="read"></a>
 </div>

In [ ]:
  # Set up directory
os.chdir("/content/drive/Shareddrives/FIT5196-s2-2022/A2/input_data/")

# Read csv
dirty = pd.read_csv("32973381_dirty_data.csv")
missing = pd.read_csv("32973381_missing_data.csv")
outlier = pd.read_csv("32973381_outlier_data.csv")

# read supplementary files
os.chdir("/content/drive/Shareddrives/FIT5196-s2-2022/A2/supplementary_files")
branches = pd.read_csv("branches.csv")
edges = pd.read_csv("edges.csv")
nodes = pd.read_csv("nodes.csv")

-------------------------------------

<div class="alert alert-block alert-warning">

## 4. Detect and handle errors in dirty.csv <a class="anchor" name="dirty"></a>
    
</div>

In the first step, dirty.csv file will be looked into and check for anomolies.

<div class="alert alert-block alert-warning">

## 4.1 Inspecting dirty.csv <a class="anchor" name="i_dirty"></a>
    
</div>

As it was told, each file contains only one type of data anomoly. Therefore as it is shown below, dirty.csv does not contain any missing values in the dataset, and it is also not expected to see any outliers value. All it contains are error values.

It is also given that the following columns are error-free:

"order_id", "time", numeric quantity in "order_items", and "delivery_fee"

In [ ]:
dirty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   order_id                 500 non-null    object 
 1   date                     500 non-null    object 
 2   time                     500 non-null    object 
 3   order_type               500 non-null    object 
 4   branch_code              500 non-null    object 
 5   order_items              500 non-null    object 
 6   order_price              500 non-null    float64
 7   customer_lat             500 non-null    float64
 8   customer_lon             500 non-null    float64
 9   customerHasloyalty?      500 non-null    int64  
 10  distance_to_customer_KM  500 non-null    float64
 11  delivery_fee             500 non-null    float64
dtypes: float64(5), int64(1), object(6)
memory usage: 47.0+ KB


In [ ]:
dirty.describe()

,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,493.451900,-35.379588,143.138771,0.064000,8.705028,13.766412
std,282.643394,19.357537,18.204149,0.244998,1.583120,2.460157
min,48.500000,-37.827382,-37.822572,0.000000,3.607000,5.353119
25%,275.937500,-37.818413,144.950237,0.000000,7.735750,12.618018
50%,423.450000,-37.812732,144.964177,0.000000,8.741000,13.890482
75%,664.750000,-37.806501,144.982310,0.000000,9.780250,15.299363
max,1492.500000,145.002889,145.019342,1.000000,14.143000,19.850763


In [ ]:
dirty.describe(include = ['O'])

,order_id,date,time,order_type,branch_code,order_items
count,500,500,500,500,500,500
unique,500,269,72,3,5,492
top,ORDJ10197,2021-03-28,17:17:44,Breakfast,NS,"[('Burger', 9), ('Salad', 8)]"
freq,1,6,14,173,172,2


In [ ]:
dirty.isnull().sum()

order_id                   0
date                       0
time                       0
order_type                 0
branch_code                0
order_items                0
order_price                0
customer_lat               0
customer_lon               0
customerHasloyalty?        0
distance_to_customer_KM    0
delivery_fee               0
dtype: int64

-------------------------------------

 <div class="alert alert-block alert-info">
    
### 4.2. Fixing date <a class="anchor" name="date"></a>

Given that "order_id" column is error-free, the first column to work on will be "date". 

As instructed, "date" values follow the "YYYY-MM-DD" format. All of them will be checked if they follow this format.


If a row contains date value with format other that that, they will be identified.

In [ ]:
# Create empty lists
date = []
error_date = []

for idx, row in dirty.iterrows():
  # set up regex method to match for the date format "YYYY-MM-DD"
  # If a row contains other format, error might occurs, try and except method is applied here
  try:
    dummy = re.findall(r"(\d{4}-\d{2}-\d{2})", row["date"])
    month = int(re.findall(r"\d{4}-(\d{2})-\d{2}", row["date"])[0])
    day = int(re.findall(r"\d{4}-\d{2}-(\d{2})", row["date"])[0])
  except IndexError:
    error_date.append(idx)
    print(row["date"])
  # Another type of error that might occur is that the "date" value followed the format of "YYYY-DD-MM" instead
  if month <= 12 and day <= 31:
    date.append(dummy)
  else:
    error_date.append(idx)
    print(row["date"])


Sat Dec  4 00:00:00 2021
2021-20-05
Tue Jan 12 00:00:00 2021
Sat Apr 10 00:00:00 2021
2021-14-03
2021-14-10


So, it was noticed that there are two types of anomolies exist in the dataset:

1. The date value was in alphabetical letter
2. The date value was in the format of "YYYY-DD-MM"

To fix this, the same iteration could be applied.

In [ ]:
# Set up dictionary indicating the numerical value for each month represents
month_dict = {"Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04", "May": "05", "Jun": "06", "Jul": "07", "Aug": "08", "Sep": "09", "Oct": "10", "Nov": "11", "Dec": "12"}

# Apply the same iteration and fix the error this time
for idx, row in dirty.iterrows():
  # Get the values for year, month, and date
  try:
    month = re.findall(r"\d{4}-(\d{2})-\d{2}", row["date"])[0]
    day = re.findall(r"\d{4}-\d{2}-(\d{2})", row["date"])[0]
    year = re.findall(r"(\d{4})-\d{2}-\d{2}", row["date"])[0]
  # Reference month_dict if the original date value contains alphabetical letter, and rearrange the data value in the desired format
  # regex methods were set up to find the corresponding values for year, month, and day
  except IndexError:
    month = re.findall(r"^\w+ (\w+)\s+\d+ .*?\s20\d+", row["date"])[0]
    month = month_dict[month]
    day = re.findall(r"^\w+\s\w+\s+(\d+)\s.*?\s20\d+", row["date"])[0]
    if len(day) < 2:
      day = f"0{day}"
    year = re.findall(r"^\w+\s\w+\s+\d+\s.*?\s(20\d+)", row["date"])[0]
    dirty.at[idx, "date"] = (f"{year}-{month}-{day}")  # Replacing error value
  # If the original data value was in "YYYY-DD-MM" format, simply swap theh "month" and "day" values
  if int(month) > 12:
    dirty.at[idx, "date"] = f"{year}-{day}-{month}"  # Replacing error value

And the "date" values are now restored to a desirable format.

In [ ]:
dirty.loc[error_date,]

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
25,ORDJ05523,2021-12-04,19:09:17,Dinner,TP,"[('Fish&Chips', 9), ('Shrimp', 6), ('Pasta', 10)]",914.00,-37.817368,144.959683,0,8.000,13.643857
132,ORDX02182,2021-05-20,12:33:48,Lunch,BK,"[('Burger', 9), ('Fries', 2), ('Salad', 7)]",423.40,-37.814511,144.938358,0,9.867,15.763245
168,ORDJ07995,2021-01-12,08:20:16,Breakfast,TP,"[('Eggs', 7), ('Pancake', 10), ('Cereal', 3), ...",473.50,-37.812870,144.942999,1,9.833,6.565756
286,ORDJ04718,2021-04-10,11:12:40,Breakfast,TP,"[('Pancake', 3), ('Cereal', 3), ('Coffee', 2),...",200.75,-37.824521,144.945817,0,8.955,13.129075
360,ORDA10577,2021-03-14,10:21:58,Breakfast,BK,"[('Coffee', 5), ('Cereal', 4), ('Pancake', 4),...",322.50,-37.801954,144.987372,0,5.998,13.650614
422,ORDZ06857,2021-10-14,17:27:53,Dinner,NS,"[('Pasta', 1), ('Shrimp', 6)]",351.50,-37.810393,144.964266,0,7.497,12.834259


-------------------------------------

<div class="alert alert-block alert-warning">
    
## 4.3 Fixing order_type <a class="anchor" name="o_type"></a>
 </div>

It is known that "order_type" value ("Breakfast", "Lunch", "Dinner") follows a specific time range. 

Given that "time" value of each row is error-free, by inspecting the "time" value, "order_type" could be restored to their correct value.

In [ ]:
# Iterate through rows which are labeled as "Breakfast", "Lunch", or "Dinner" separately.
# Due to the fact that "time" value is always correct, correct "order_type" by referencing the time of the record

# Chceking for Breakfast order_type
for idx, row in dirty[dirty['order_type'] == 'Breakfast'].iterrows():
  if row["time"] > '12:00:00' and row["time"] <= '16:00:00':
    dirty.loc[idx, "order_type"] = ["Lunch"]
  if row["time"] > "16:00:00" and row["time"] <= "20:00:00":
    dirty.loc[idx, "order_type"] = ["Dinner"]

# Chceking for Lunch order_type
for idx, row in dirty[dirty['order_type'] == 'Lunch'].iterrows():
  if row["time"] >= '08:00:00' and row["time"] <= '12:00:00':
    dirty.loc[idx, "order_type"] = ["Breakfast"]
  if row["time"] > "16:00:00" and row["time"] <= "20:00:00":
    dirty.loc[idx, "order_type"] = ["Dinner"]

# Chceking for Dinner order_type
for idx, row in dirty[dirty["order_type"] == "Dinner"].iterrows():
  if row["time"] >= '08:00:00' and row["time"] <= '12:00:00':
    dirty.loc[idx, "order_type"] = ["Breakfast"]
  if row["time"] > '12:00:00' and row["time"] <= '16:00:00':
    dirty.loc[idx, "order_type"] = ["Lunch"]


-------------------------------------

<div class="alert alert-block alert-warning">
    
## 4.4 Fixing branch_code <a class="anchor" name="br_code"></a>
 </div>

Some "branch_code" values in the dataset are in lower case letter.
These could be quickly fixed by setting all "branch_code" value as upper case letter.

In [ ]:
# Get value counts for branch_code column
dirty["branch_code"].value_counts()

NS    172
BK    164
TP    162
tp      1
bk      1
Name: branch_code, dtype: int64

In [ ]:
# Revised all values in branch_code column as uppercase lette 
for idx, row in dirty.iterrows():
  dirty.at[idx, 'branch_code'] = dirty.at[idx, 'branch_code'].upper()

# revisit value counts for branch_code column
dirty["branch_code"].value_counts()

NS    172
BK    165
TP    163
Name: branch_code, dtype: int64

-------------------------------------

<div class="alert alert-block alert-warning">
    
## 4.5 Fixing order_item <a class="anchor" name="o_item"></a>
 </div>

Given that each meal has a distinct set of items in the menu, items that do not belong to a specific meal type should be revised.

As the dirty dataset contains error values which could notn be relied on, other dataset: missing and outliers should be taken as a reference for such issue, in order to obtain the correct values.

In the following cell, the outlier.csv file is inspected, as there are no missing value in it, and the only anomolies in Outlier dataframe is in delivery_fee column. This could be a good reference to apply on the dirty data.

In [ ]:
outlier.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   order_id                 500 non-null    object 
 1   date                     500 non-null    object 
 2   time                     500 non-null    object 
 3   order_type               500 non-null    object 
 4   branch_code              500 non-null    object 
 5   order_items              500 non-null    object 
 6   order_price              500 non-null    float64
 7   customer_lat             500 non-null    float64
 8   customer_lon             500 non-null    float64
 9   customerHasloyalty?      500 non-null    int64  
 10  distance_to_customer_KM  500 non-null    float64
 11  delivery_fee             500 non-null    float64
dtypes: float64(5), int64(1), object(6)
memory usage: 47.0+ KB


Storing items to their corresponding meal types

In [ ]:
# Creating lists to store items for each order_type
breakfast_items = []
lunch_items = []
dinner_items = []

for idx, row in outlier.iterrows():
  # Set up regex method to find all meal items
  items = (re.findall(r'\((.*?),.*?\)', row["order_items"]))
  # Store item into their corresponing order_type's list
  for n in range(len(items)):
    if row["order_type"] == "Breakfast":
      breakfast_items.append(items[n])
    elif row["order_type"] == "Lunch":
      lunch_items.append(items[n])
    else:
      dinner_items.append(items[n])

# Apply the "set" method to remove duplicate values
breakfast_set = sorted(list(set(breakfast_items)))
lunch_set = sorted(list(set(lunch_items)))
dinner_set = sorted(list(set(dinner_items)))

In [ ]:
breakfast_set

["'Cereal'", "'Coffee'", "'Eggs'", "'Pancake'"]

In [ ]:
lunch_set

["'Burger'", "'Chicken'", "'Fries'", "'Salad'", "'Steak'"]

In [ ]:
dinner_set

["'Fish&Chips'", "'Pasta'", "'Salmon'", "'Shrimp'"]

After obtaining the items for each order_type, outlier.csv file has finished its job in this taks.

Dirty.csv will be revisited, trying to fix the error values in "order_items" column.

In [ ]:
# categorise Dirty dataframe into three subsets, grouping by order_type
breakfast_dirty = dirty[dirty["order_type"] == "Breakfast"]
lunch_dirty = dirty[dirty["order_type"] == "Lunch"]
dinner_dirty = dirty[dirty["order_type"] == "Dinner"]

-------------------------------------

<div class="alert alert-block alert-warning">
    
## 4.5.1 Fixing Breakfast items <a class="anchor" name="breakfast_item"></a>
 </div>

To fix order_item in breakfast subset

In [ ]:
# Create list to obtain indexes with error values
error_item = []

for idx, row in breakfast_dirty.iterrows():
  # Set up regex method to find all items in the breakfast subset
  items = (re.findall(r'\((.*?),.*?\)', row["order_items"]))
  for n in range(len(items)):
    # Export row index if an item is found that is not on the breakfast menu
    if items[n] not in breakfast_items:
      error_item.append(idx)

In [ ]:
dirty.loc[error_item,]

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
156,ORDB10459,2021-03-03,10:01:41,Breakfast,TP,"[('Shrimp', 3), ('Pancake', 6)]",196.50,-37.815824,144.932692,0,11.011,12.792280
260,ORDK07024,2021-01-23,08:40:33,Breakfast,BK,"[('Coffee', 7), ('Cereal', 10), ('Chicken', 3)...",473.25,-37.814731,144.965252,0,7.598,15.309942


Now that all errors are found, a method has to be set up to find the correct item in the "order_items" column.

Given that a row can only contain one type of anomoly, and the numeric values in "order_item" column are always correct. The "order_price" column could be referenced in this case.

An equation could be set up as:

Item1_price * item1_quantity + item2_price * item2_quantity = order_price

By knowing the prices of all items, the correct value for error item could now be restored.

Therefore, other rows in the data set could be taken as reference.

However, due to the fact that other row may also contain errors in "order_items" or "order_price" column. In this case, two samples will be taken to verify the findings.

In [ ]:
# Create an empty dummy list
dummy = []

# As there are four itmes on the breakfast menu, four samples which contain all four items have to be taken to solve the linear algebra equation
for idx, row in breakfast_dirty.iterrows():
  if len(re.findall(r'\((.*?),.*?\)', row["order_items"])) == 4:
    dummy.append(idx)

# Randomly taking 4 records for each sample
sample1 = dirty.loc[dummy,].tail(4)[["order_items","order_price"]]
sample2 = dirty.loc[dummy,].head(4)[["order_items","order_price"]]

In [ ]:
# Creating empty lists
input = []
output = []

for idx, row in sample1.iterrows():
  # set up regex method to store the number of orders for items for sample, and store values as integer
  dummy = (re.findall(r"\('Cereal', (\d+)\)", row["order_items"]))
  dummy.append(re.findall(r"\('Coffee', (\d+)\)", row["order_items"])[0])
  dummy.append(re.findall(r"\('Eggs', (\d+)\)", row["order_items"])[0])
  dummy.append(re.findall(r"\('Pancake', (\d+)\)", row["order_items"])[0])
  for n in range(len(dummy)):
    dummy[n] = int(dummy[n])
  input.append(dummy)
  # Also storing the order_prices of the sample taken
  output.append(row["order_price"])

# As such, four equations were obtained
# Apply np.linalg library to solve the equations, and roundn up solutions to 2 decimal places
sol1 = np.linalg.solve(input, output)
for n in range(len(sol1)):
  sol1[n] = round(sol1[n], 2)

# Apply the exact same method to the other sample
input = []
output = []
for idx, row in sample2.iterrows():
  dummy = (re.findall(r"\('Cereal', (\d+)\)", row["order_items"]))
  dummy.append(re.findall(r"\('Coffee', (\d+)\)", row["order_items"])[0])
  dummy.append(re.findall(r"\('Eggs', (\d+)\)", row["order_items"])[0])
  dummy.append(re.findall(r"\('Pancake', (\d+)\)", row["order_items"])[0])
  for n in range(len(dummy)):
    dummy[n] = int(dummy[n])
  input.append(dummy)
  output.append(row["order_price"])

sol2 = np.linalg.solve(input, output)
for n in range(len(sol2)):
  sol2[n] = round(sol2[n], 2)

In [ ]:
# Check for item price obtained from the two samples
print(sol1 == sol2)

print(sol1)

# Creat a dictionary for items to prices, as well as a inverted dictionary for prices to items, which both will be applied in the following tasks
price_dict = dict(zip(breakfast_set, sol1))
inv_price_dict = dict(zip(sol1, breakfast_set))

[ True  True  True  True]
[20.    5.5  21.   22.25]


Notice that the prices are listed in the order as the regex methods set up

So that we could obtain the price of each breakfast items:

Cereal: $20 


Coffee: $5.5


Eggs: $21


Pancake: $22.25

By inspecting the item price of the unknown item, the quantity ordered, the price and quantity of other items ordered, as well as the total order_price, the unknown item could now be found by referencing the inverted prices to items dictionary 

In [ ]:
# Revisit the row which contains error values in "order_item" column for "Breakfast" order type
for idx, row in dirty.loc[error_item,].iterrows():
  # Use regex methods to find name of item and the quantity of item
  items = re.findall(r'\((.*?),.*?\)', row["order_items"])
  quantities = re.findall(r'\(.*?, (\d+)\)', row["order_items"])
  # For item that does not belong to the "Breakfast" order type
  # As the item is unknown while the quantity of that item ordered is correct
  # The inverted dictionary for prices to items could be applied, to find the item by referencing to its price
  for item in items:
    if item not in breakfast_set:
      quan = quantities[items.index(item)]
      quantities.remove(quan)
      items.remove(item)
      products = []
      # Apply price_dict and inv_price_dict
      for q, i in zip(quantities, items):
        products.append(int(q) * price_dict[i])
      # Obtain item price from the equation set up
      item_price = ((row["order_price"] - sum(products)) / int(quan))
      # By reference the item_price to the inverted dictionary, the revised item could be found
      rev_item = inv_price_dict[item_price]
      dirty.at[idx,"order_items"] = re.sub(item, rev_item, dirty.loc[idx,"order_items"])  # Replace error values to the correct value

In [ ]:
# order_items are now rewrite as the correct value
dirty.loc[error_item,]

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
156,ORDB10459,2021-03-03,10:01:41,Breakfast,TP,"[('Eggs', 3), ('Pancake', 6)]",196.50,-37.815824,144.932692,0,11.011,12.792280
260,ORDK07024,2021-01-23,08:40:33,Breakfast,BK,"[('Coffee', 7), ('Cereal', 10), ('Pancake', 3)...",473.25,-37.814731,144.965252,0,7.598,15.309942


-------------------------------------

<div class="alert alert-block alert-warning">
    
## 4.5.2 Fixing Lunch items <a class="anchor" name="lunch_item"></a>
 </div>

After fixing order_item for "Breakfast" order_type, the same method will be applied to "Lunch" order_type with some variation due to the difference in items and format.

In [ ]:
# Finding all rows with error value
error_item = []

# Iterate through lunch subset
for idx, row in lunch_dirty.iterrows():
  # Obtain lunch items in each row
  items = (re.findall(r'\((.*?),.*?\)', row["order_items"]))
  # Find items that do not exist in lunch_menu
  for n in range(len(items)):
    if items[n] not in lunch_items:
      error_item.append(idx)

In [ ]:
dirty.loc[error_item,]

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
285,ORDX07999,2021-06-26,13:44:47,Lunch,BK,"[('Fish&Chips', 1), ('Chicken', 9), ('Salad', 7)]",420.4,-37.812519,145.013206,0,3.607,11.910820
332,ORDX05200,2021-06-07,13:14:21,Lunch,BK,"[('Chicken', 3), ('Coffee', 5), ('Salad', 1), ...",550.2,-37.814924,144.945104,0,9.150,15.018921
468,ORDC05947,2021-05-15,12:23:39,Lunch,NS,"[('Fish&Chips', 8), ('Fries', 7), ('Chicken', ...",889.0,-37.813124,144.963099,0,7.927,15.295070


In [ ]:
# Empty dummy list
dummy = []

# As there are 5 items on lunch menu, 5 equations need to be set up, each containing all 5 items
for idx, row in lunch_dirty.iterrows():
  if len(re.findall(r'\((.*?),.*?\)', row["order_items"])) == 5:
    dummy.append(idx)

# Taking two samples
sample1 = dirty.loc[dummy,].tail(5)[["order_items","order_price"]]
sample2 = dirty.loc[dummy,].head(5)[["order_items","order_price"]]

In [ ]:
# Inspect lunch_set's items
# 
lunch_set

["'Burger'", "'Chicken'", "'Fries'", "'Salad'", "'Steak'"]

In [ ]:
# Apply the same method to obtain prices for items on lunch menu

# Create empty lists
input = []
output = []
for idx, row in sample1.iterrows():
  # Set up regex methods for order quantity for each item in the first sample
  dummy = (re.findall(r"\('Burger', (\d+)\)", row["order_items"]))
  dummy.append(re.findall(r"\('Chicken', (\d+)\)", row["order_items"])[0])
  dummy.append(re.findall(r"\('Fries', (\d+)\)", row["order_items"])[0])
  dummy.append(re.findall(r"\('Salad', (\d+)\)", row["order_items"])[0])
  dummy.append(re.findall(r"\('Steak', (\d+)\)", row["order_items"])[0])
  for n in range(len(dummy)):
    dummy[n] = int(dummy[n])
  input.append(dummy)
  # Also obtain total order price for orders
  output.append(row["order_price"])

# Solve for item prices and round up by 2 d.p. for first sample
sol1 = np.linalg.solve(input, output)
for n in range(len(sol1)):
  sol1[n] = round(sol1[n], 2)

# Same for the second sample
input = []
output = []
for idx, row in sample2.iterrows():
  dummy = (re.findall(r"\('Burger', (\d+)\)", row["order_items"]))
  dummy.append(re.findall(r"\('Chicken', (\d+)\)", row["order_items"])[0])
  dummy.append(re.findall(r"\('Fries', (\d+)\)", row["order_items"])[0])
  dummy.append(re.findall(r"\('Salad', (\d+)\)", row["order_items"])[0])
  dummy.append(re.findall(r"\('Steak', (\d+)\)", row["order_items"])[0])
  for n in range(len(dummy)):
    dummy[n] = int(dummy[n])
  input.append(dummy)
  output.append(row["order_price"])

# Solve for item prices and round up by 2 d.p. for the second sample
sol2 = np.linalg.solve(input, output)
for n in range(len(sol2)):
  sol2[n] = round(sol2[n], 2)

In [ ]:
# Verify solution
print(sol1 == sol2)
print(sol1)

[ True  True  True  True  True]
[31.  32.  12.  17.2 45. ]


In [ ]:
# Adding item and item prices obtained to the two dictionaries created earlier
for i in range(len(sol1)):
  price_dict[lunch_set[i]] = sol1[i]
  inv_price_dict[sol1[i]] = lunch_set[i]

In [ ]:
# Revisit the dirty dataframe for "Lunch" order type
# Apply the same method applied to "Breakfast" order type to correct error items
for idx, row in dirty.loc[error_item,].iterrows():
  items = re.findall(r'\((.*?),.*?\)', row["order_items"])
  quantities = re.findall(r'\(.*?, (\d+)\)', row["order_items"])
  for item in items:
    if item not in lunch_set:
      quan = quantities[items.index(item)]
      quantities.remove(quan)
      items.remove(item)
      products = []
      for q, i in zip(quantities, items):
        products.append(int(q) * price_dict[i])
      item_price = round((row["order_price"] - sum(products)) / int(quan), 2)
      rev_item = inv_price_dict[item_price]
      dirty.at[idx,"order_items"] = re.sub(item, rev_item, dirty.loc[idx,"order_items"])  # Replace error value with the correct value

In [ ]:
# order_items are now rewrite as the correct value for "Lunch" order_type
dirty.loc[error_item,]

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
285,ORDX07999,2021-06-26,13:44:47,Lunch,BK,"[('Fries', 1), ('Chicken', 9), ('Salad', 7)]",420.4,-37.812519,145.013206,0,3.607,11.910820
332,ORDX05200,2021-06-07,13:14:21,Lunch,BK,"[('Chicken', 3), ('Burger', 5), ('Salad', 1), ...",550.2,-37.814924,144.945104,0,9.150,15.018921
468,ORDC05947,2021-05-15,12:23:39,Lunch,NS,"[('Burger', 8), ('Fries', 7), ('Chicken', 5), ...",889.0,-37.813124,144.963099,0,7.927,15.295070


-------------------------------------

<div class="alert alert-block alert-warning">
    
## 4.5.3 Fixing Dinner items <a class="anchor" name="dinner_item"></a>
 </div>

And the same logic and method could be applied to Dinner order type as well

In [ ]:
# Get rows with errors
error_item = []
for idx, row in dinner_dirty.iterrows():
  items = row["order_items"]
  items = (re.findall(r'\((.*?),.*?\)', items))
  for n in range(len(items)):
    if items[n] not in dinner_items:
      error_item.append(idx)

In [ ]:
dirty.loc[error_item, ]

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
18,ORDC09537,2021-09-06,16:16:54,Dinner,NS,"[('Pancake', 4), ('Fish&Chips', 3), ('Shrimp',...",702.0,-37.802362,144.991536,0,8.306,14.390964
428,ORDX03837,2021-11-01,18:08:27,Dinner,BK,"[('Burger', 7), ('Fish&Chips', 4), ('Salmon', ...",846.5,-37.811933,144.963622,0,7.540,14.454771


In [ ]:
# Obtain samples
dummy = []
for idx, row in dinner_dirty.iterrows():
  if len(re.findall(r'\((.*?),.*?\)', row["order_items"])) == 4:
    dummy.append(idx)
sample1 = dirty.loc[dummy,].tail(4)[["order_items","order_price"]]
sample2 = dirty.loc[dummy,].head(4)[["order_items","order_price"]]

Due to a problem that the second sample contains error value itself, it could not be involved in our calculation.

A third sample is taken instead.

In [ ]:
random.seed(1)  # Control randomness
# Get random sample
sample3 = dirty.loc[random.choices(dummy, k = 4),][["order_items", "order_price"]]

In [ ]:
# And sample 3 is an eligible sample for calculation
sample3

,order_items,order_price
94,"[('Pasta', 5), ('Shrimp', 10), ('Salmon', 9), ...",1221.5
394,"[('Pasta', 10), ('Shrimp', 6), ('Salmon', 3), ...",1072.0
337,"[('Shrimp', 1), ('Salmon', 6), ('Fish&Chips', ...",732.5
138,"[('Shrimp', 9), ('Fish&Chips', 5), ('Salmon', ...",1085.0


In [ ]:
dinner_set

["'Fish&Chips'", "'Pasta'", "'Salmon'", "'Shrimp'"]

In [ ]:
# Obtain prices for items on dinner menu
input = []
output = []
for idx, row in sample1.iterrows():
  dummy = (re.findall(r"\('Fish&Chips', (\d+)\)", row["order_items"]))
  dummy.append(re.findall(r"\('Pasta', (\d+)\)", row["order_items"])[0])
  dummy.append(re.findall(r"\('Salmon', (\d+)\)", row["order_items"])[0])
  dummy.append(re.findall(r"\('Shrimp', (\d+)\)", row["order_items"])[0])
  for n in range(len(dummy)):
    dummy[n] = int(dummy[n])
  input.append(dummy)
  output.append(row["order_price"])

# Solve for item prices and round up by 2 d.p. for first sample
sol1 = np.linalg.solve(input, output)
for n in range(len(sol1)):
  sol1[n] = round(sol1[n], 2)

input = []
output = []
for idx, row in sample3.iterrows():
  dummy = (re.findall(r"\('Fish&Chips', (\d+)\)", row["order_items"]))
  dummy.append(re.findall(r"\('Pasta', (\d+)\)", row["order_items"])[0])
  dummy.append(re.findall(r"\('Salmon', (\d+)\)", row["order_items"])[0])
  dummy.append(re.findall(r"\('Shrimp', (\d+)\)", row["order_items"])[0])
  for n in range(len(dummy)):
    dummy[n] = int(dummy[n])
  input.append(dummy)
  output.append(row["order_price"])

# Solve for item prices and round up by 2 d.p. for the third sample
sol3 = np.linalg.solve(input, output)
for n in range(len(sol3)):
  sol3[n] = round(sol3[n], 2)

In [ ]:
# Verfify solutions
print(sol1 == sol3)
print(sol1)

[ True  True  True  True]
[35.  27.5 41.  54. ]


In [ ]:
# Adding items and item prices to the two dictionaries
for i in range(len(sol1)):
  price_dict[dinner_set[i]] = sol1[i]
  inv_price_dict[sol1[i]] = dinner_set[i]

In [ ]:
# Revisit the dirty dataframe for "Dinner" order type
# Apply the same method applied to the other two order types to correct error items
for idx, row in dirty.loc[error_item,].iterrows():
  items = re.findall(r'\((.*?),.*?\)', row["order_items"])
  quantities = re.findall(r'\(.*?, (\d+)\)', row["order_items"])
  for item in items:
    if item not in dinner_set:
      quan = quantities[items.index(item)]
      quantities.remove(quan)
      items.remove(item)
      products = []
      for q, i in zip(quantities, items):
        products.append(int(q) * price_dict[i])
      item_price = round((row["order_price"] - sum(products)) / int(quan), 2)
      rev_item = inv_price_dict[item_price]
      dirty.at[idx,"order_items"] = re.sub(item, rev_item, dirty.loc[idx,"order_items"]) # Replace error items with the correct item

In [ ]:
# Inspect rows with errors before
dirty.loc[error_item, ]

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
18,ORDC09537,2021-09-06,16:16:54,Dinner,NS,"[('Salmon', 4), ('Fish&Chips', 3), ('Shrimp', ...",702.0,-37.802362,144.991536,0,8.306,14.390964
428,ORDX03837,2021-11-01,18:08:27,Dinner,BK,"[('Shrimp', 7), ('Fish&Chips', 4), ('Salmon', ...",846.5,-37.811933,144.963622,0,7.540,14.454771


-------------------------------------

<div class="alert alert-block alert-warning">
    
## 4.6 Fixing order_price <a class="anchor" name="o_price"></a>
 </div>

By obtaining a dictionary for all items on every type of menu, now the order_price column could be fixed as well.

Assuming the methods in 4.5 are conducted correctly, then order_items should contain no error value anymore. 

By referencing to the price_dict dictionary as well as each food item and the quantity ordered, the correct order price could now be computed. 

In [ ]:
# Price dictionary created

price_dict

{"'Cereal'": 20.0,
 "'Coffee'": 5.5,
 "'Eggs'": 21.0,
 "'Pancake'": 22.25,
 "'Burger'": 31.0,
 "'Chicken'": 32.0,
 "'Fries'": 12.0,
 "'Salad'": 17.2,
 "'Steak'": 45.0,
 "'Fish&Chips'": 35.0,
 "'Pasta'": 27.5,
 "'Salmon'": 41.0,
 "'Shrimp'": 54.0}

In [ ]:
# Iterate through every row in the dirty dataframe
for idx, row in dirty.iterrows():
  # Set up regex methods for food items and the quantity ordered
  items = re.findall(r'\((.*?),.*?\)', row["order_items"])
  quantities = re.findall(r'\(.*?, (\d+)\)', row["order_items"])
  products = []
  # Compute for the correct order price
  for q, i in zip(quantities, items):
    products.append(int(q) * price_dict[i])
  # If the computed order price differ from the original order price in the dirty dataframe, replace the original data with the computed value
  if sum(products) != row["order_price"]:
    dirty.at[idx, "order_price"] = round(sum(products), 2)  # replace error value with the correct value
  


-------------------------------------

<div class="alert alert-block alert-warning">
    
## 4.7 Fixing Latitude and Longitude values <a class="anchor" name="latlon"></a>
 </div>

By using the describe method for dirty dataframw, it is noticed that there are extreme values for "customer_lat" and "customer_lon" columns.

By the look of it, which the maximum value for "customer_lat" is around 145, while the average is -35, and the minimum value for "customer_lon" is -37 while the average is 143, it is possible that the value for "customer_lat" and "customer_lon" are mixed up for these rows.

In [ ]:
# Checking for values in "customer_lat" and "customer_lon" columns
dirty.describe()

,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,495.294200,-35.379588,143.138771,0.064000,8.705028,13.766412
std,281.959472,19.357537,18.204149,0.244998,1.583120,2.460157
min,48.500000,-37.827382,-37.822572,0.000000,3.607000,5.353119
25%,279.250000,-37.818413,144.950237,0.000000,7.735750,12.618018
50%,423.750000,-37.812732,144.964177,0.000000,8.741000,13.890482
75%,680.250000,-37.806501,144.982310,0.000000,9.780250,15.299363
max,1492.500000,145.002889,145.019342,1.000000,14.143000,19.850763


In [ ]:
# Inspect rows with "customer_lon" value smaller than 144
dirty[dirty["customer_lon"] < 144]

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
99,ORDX08444,2021-03-05,10:01:41,Breakfast,BK,"[('Cereal', 6), ('Eggs', 10), ('Pancake', 1), ...",390.75,145.002889,-37.803977,0,5.165,10.391682
229,ORDY02957,2021-10-31,17:58:18,Dinner,TP,"[('Fish&Chips', 2), ('Pasta', 3), ('Salmon', 9)]",521.50,144.960983,-37.822572,0,7.584,13.883925
258,ORDB05947,2021-12-31,20:00:00,Dinner,TP,"[('Pasta', 9), ('Salmon', 5), ('Fish&Chips', 1)]",487.50,144.975910,-37.809762,0,9.169,12.565660
299,ORDI00914,2021-09-21,16:37:10,Dinner,NS,"[('Shrimp', 7), ('Pasta', 3), ('Salmon', 9)]",829.50,144.996888,-37.804224,0,8.754,14.607972
309,ORDY03133,2021-09-28,16:57:27,Dinner,TP,"[('Salmon', 1), ('Fish&Chips', 4), ('Pasta', 10)]",456.00,144.967498,-37.803898,0,9.547,13.897038


From the above dataframe, it is noticed that "customer_lat" values are all around 144 to 145, while "customer_lon" values are around -37. 

The assumption that they are mixed up could be correct.

By swapping the values between "customer_lat" and "customer_lon" for these rows could make sense of the dataset.

In [ ]:
# Swapping the two "cust_lat" and "cust_lon" values for the rows in the above dataframe
for idx, row in dirty[dirty["customer_lon"] < 144].iterrows():
  if row["customer_lon"] < 0 and row["customer_lat"] >144:
    dirty.at[idx, "customer_lon"], dirty.at[idx, "customer_lat"] = row["customer_lat"], row["customer_lon"]  # Swapping values

In [ ]:
# No more rows with unreasonable "customer_lon" value
dirty[dirty["customer_lon"] < 144]

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee


Next, "customer_lat" values are looked into. Due to the fact that the records are the data for a restaurant in Melbourne, it is unreasonable to have delivery location latitude values that are so far off.

In [ ]:
# Inspect for extreme values in "customer_lat" column
dirty[dirty["customer_lat"] > -37]

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
15,ORDI09273,2021-10-24,17:48:10,Dinner,NS,"[('Shrimp', 1), ('Salmon', 8), ('Fish&Chips', 2)]",452.0,37.814507,144.999935,0,9.891,18.026639
234,ORDC06289,2021-08-13,15:26:11,Lunch,NS,"[('Fries', 3), ('Burger', 10), ('Chicken', 5),...",810.4,37.798083,144.987543,0,7.962,13.474298
256,ORDX01733,2021-07-24,14:45:38,Lunch,BK,"[('Steak', 2), ('Burger', 1), ('Chicken', 2), ...",291.4,37.820319,144.957620,0,8.345,16.337138
425,ORDX05709,2021-03-06,10:01:41,Breakfast,BK,"[('Eggs', 4), ('Cereal', 6)]",204.0,37.806093,144.954451,0,8.230,15.974455


The error happened in "customer_lat" is that the values are missing the negative signs, while the numbers are actually reasonably indicating Melbourne area if a negative sign is added.

A possible fix is to add back a negative sign for these values to make sense of the data.

In [ ]:
# Adding negative sign back to these error value
for idx, row in dirty[dirty["customer_lat"] > -37].iterrows():
  dirty.at[idx, "customer_lat"] = -row["customer_lat"]

# So that values in "customer_lat" column make sense now 
dirty[dirty["customer_lat"] > -37]

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee


In [ ]:
dirty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   order_id                 500 non-null    object 
 1   date                     500 non-null    object 
 2   time                     500 non-null    object 
 3   order_type               500 non-null    object 
 4   branch_code              500 non-null    object 
 5   order_items              500 non-null    object 
 6   order_price              500 non-null    float64
 7   customer_lat             500 non-null    float64
 8   customer_lon             500 non-null    float64
 9   customerHasloyalty?      500 non-null    int64  
 10  distance_to_customer_KM  500 non-null    float64
 11  delivery_fee             500 non-null    float64
dtypes: float64(5), int64(1), object(6)
memory usage: 47.0+ KB


-------------------------------------

<div class="alert alert-block alert-warning">
    
## 4.8 Fixing distance to customer column <a class="anchor" name="dist"></a>
 </div>

Next, "distance_to_customer" column will be checked.

As customer loyalty is closely related to delivery fee of an order, which the latter is related to the delivery distance, given that "delivery_fee" column is error-free. Therefore, in order to fix the column of customer loyalty, distance to cutsomer need to be handled first.

Given that the restaurant adopts the Djikstra algorithm to compute for the shortest distance between customer and restaurant, supplementary files containing nodes, edges and branches information willl come in use.

In [ ]:
# Inspecting branches
branches

,branch_code,branch_name,branch_lat,branch_lon
0,NS,Nickolson,-37.773803,144.983647
1,TP,Thompson,-37.861835,144.905716
2,BK,Bakers,-37.815834,145.046450


In [ ]:
# Inspecting nodes
nodes.head(5)

,node,lat,lon
0,711327755,-37.807675,144.955873
1,711327756,-37.807094,144.955978
2,711327759,-37.807301,144.957817
3,711327760,-37.807885,144.957719
4,777781264,-37.805221,144.952173


In [ ]:
# Any branch is one of the nodes in nodes dataframe as well
nodes[(nodes["lon"] == branches.loc[0, "branch_lon"]) & (nodes["lat"] == branches.loc[0, "branch_lat"])]

,node,lat,lon
2036,2455254505,-37.773803,144.983647


In [ ]:
# Mergeing branches and nodes dataframes to obtain node id for all branches
# Also transform branches dataframe format to one that is more similar to that of nodes dataframe
branches = branches.merge(nodes, how = "left", left_on = ["branch_lat", "branch_lon"], right_on = ["lat", "lon"])[["branch_code", "node", "lat", "lon"]]

In [ ]:
# New branches dataframe
branches

,branch_code,node,lat,lon
0,NS,2455254505,-37.773803,144.983647
1,TP,1390575046,-37.861835,144.905716
2,BK,1889485053,-37.815834,145.046450


In [ ]:
# and every customer location should be one of the nodes too
# Taking one sample for checking purpose
nodes[(nodes["lat"] == dirty.loc[1, "customer_lat"]) | (nodes["lon"] == dirty.loc[1, "customer_lon"])]

,node,lat,lon
12148,307587086,-37.81491,144.958935


In [ ]:
# create a subset of customer location, together with the node id
dest = dirty[["order_id", "customer_lat", "customer_lon", "distance_to_customer_KM"]]
dest = dest.merge(nodes, how = "left", left_on = ["customer_lat", "customer_lon"], right_on = ["lat", "lon"])

In [ ]:
# Check for missing values
# If there are no missing values, this gives strong evidence that the corrections for customer_lat and customer_lon were performed correctly 
df = dest[dest['node'].isnull()]
df.info()  

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   order_id                 0 non-null      object 
 1   customer_lat             0 non-null      float64
 2   customer_lon             0 non-null      float64
 3   distance_to_customer_KM  0 non-null      float64
 4   node                     0 non-null      int64  
 5   lat                      0 non-null      float64
 6   lon                      0 non-null      float64
dtypes: float64(5), int64(1), object(1)
memory usage: 0.0+ bytes


In [ ]:
# Final subset created that contains delivery information, including order id, node id, latitude, longitude and distance between the branch and destination
dest = dest[["order_id", "node", "lat", "lon", "distance_to_customer_KM"]]
dest

,order_id,node,lat,lon,distance_to_customer_KM
0,ORDJ10197,60092011,-37.822350,144.984071,9.354
1,ORDX10901,307587086,-37.814910,144.958935,11.286
2,ORDY01694,4638961335,-37.809739,144.946342,9.720
3,ORDJ09900,4283908143,-37.809609,144.975625,9.065
4,ORDK08694,781092635,-37.802293,144.942399,9.377
...,...,...,...,...,...
495,ORDI06666,534017209,-37.809713,144.987961,8.396
496,ORDK06810,3218226998,-37.806341,144.945380,9.122
497,ORDA06305,846566359,-37.823332,144.969914,7.982
498,ORDX08700,1449431612,-37.815974,144.972464,6.940


As all start points (branches) and destinations (customers' location) are obtained, a graph could be built based on the nodes, as well as the edges provided.

Once a graph is set up, the distance between the starting point of an order (a branch of the restaurant) and the destination (customer location) could now be computed.

In [ ]:
# Create empty graph and add nodes and edges
G = nx.Graph()

# adding nodes
for i in nodes["node"]:
  G.add_node(i)

# adding edges with weight
for idx, row in edges.iterrows():
  u = row['u']
  v = row['v']
  distance = row["distance(m)"] / 1000
  G.add_edge(u, v, weight = distance)

In [ ]:
# Compute distance between branch and customer location

# Create empty list
new_dist = []

# As dest is a subset of the dirty dataframe, the arrangement of indexes follow the original dataframe
for idx, row in dest[["order_id", "node"]].iterrows():
  # finding the node id for start location and destination
  start = branches[branches["branch_code"] == dirty.loc[idx, "branch_code"]].iloc[0, 1]
  destination = row["node"]
  # use method in networkx package to compute for distance
  distance = (nx.shortest_path_length(G, source = start, target = destination, weight = "weight"))
  new_dist.append(round(distance, 3)) # Store values into the empty list



In [ ]:
# Create a copy for dirty dataframe and add new column with the computed distance value
dirty_copy = dirty.copy()
dirty_copy["new_distance"] = new_dist

In [ ]:
# Finding all rows with error values, where the original distance value is different to the computed value
error_dist = []
for idx, row in dirty_copy.iterrows():
  if row["distance_to_customer_KM"] != row["new_distance"]:
    error_dist.append(idx) 

In [ ]:
# Rows with error value
dirty_copy.loc[error_dist,]

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee,new_distance
6,ORDK08311,2021-07-24,14:45:38,Lunch,NS,"[('Steak', 4), ('Burger', 2), ('Chicken', 10),...",656.40,-37.821130,144.957737,0,8.430,16.395970,9.276
67,ORDI02993,2021-04-28,11:53:14,Breakfast,NS,"[('Coffee', 3), ('Pancake', 9), ('Cereal', 2)]",256.75,-37.805846,144.927460,0,9.962,16.052133,10.891
95,ORDK10038,2021-11-02,18:08:27,Dinner,BK,"[('Pasta', 9), ('Salmon', 7)]",534.50,-37.805926,144.944006,0,7.755,16.174798,9.280
98,ORDA04099,2021-01-23,08:40:33,Breakfast,BK,"[('Coffee', 6), ('Pancake', 2)]",77.50,-37.811586,144.960760,0,9.280,15.055169,7.766
134,ORDC06312,2021-09-23,16:47:19,Dinner,NS,"[('Shrimp', 9), ('Pasta', 8), ('Fish&Chips', 6...",1203.00,-37.809566,144.983612,0,4.998,14.015641,8.054
245,ORDZ07212,2021-08-27,15:56:37,Lunch,NS,"[('Salad', 7), ('Burger', 5)]",275.40,-37.801369,145.004156,0,10.592,14.721216,9.451
273,ORDA11064,2021-05-29,12:54:05,Lunch,NS,"[('Steak', 2), ('Chicken', 4), ('Fries', 5), (...",371.00,-37.813795,144.971823,0,7.035,15.212990,7.671
294,ORDI02090,2021-04-13,11:22:49,Breakfast,TP,"[('Pancake', 1), ('Eggs', 1), ('Cereal', 2)]",83.25,-37.819338,144.985153,0,9.030,14.078133,9.212
358,ORDZ04462,2021-06-21,13:34:38,Lunch,BK,"[('Burger', 5), ('Salad', 2), ('Steak', 2), ('...",483.40,-37.805876,144.956074,0,7.940,13.936531,8.053
368,ORDC01068,2021-08-15,15:26:11,Lunch,NS,"[('Chicken', 5), ('Burger', 5), ('Steak', 6)]",585.00,-37.816269,144.956304,0,10.433,16.337049,8.714


In [ ]:
# Replace whole distance column in dirty dataframe with computed values.
# Assuming the computation is correct, distance column in dirty dataframe is now restored with correct values
dirty["distance_to_customer_KM"] = dirty_copy["new_distance"]

-------------------------------------

<div class="alert alert-block alert-warning">
    
## 4.9 Fixing Loyalty value <a class="anchor" name="loyalty"></a>
 </div>

For the Loyalty column, there are no direct indicator that gives value of a customer has loyalty discount or not.

However, it was given that a 50% discount will be given to a customer's delivery fee if a customer has loyalty.

Given that delivery fee column is error-free, and it is based on three factors:<br>
  1. If the order is on the weekend
  2. Time of the order (morning, afternoon or evening)
  3. Distance between branch and customer<br>

With these 3 factors, a linear regression model could be built to compute for a delivery fee value for each order.

If a the actual delivery fee is about half of the computed value, it is known that the customer has loyalty, and this result could be applied to corrected "customerHasloyalty?" column in the dataframe.


In order to establish a linear regression model, a training dataset has to be obtained.

Consider the following options:

1. Semi-cleaned dirty dataframe:<br>
Throughout session 4.1 - 4.8, dirty dataframe is now almost cleaned, left only with the loyalty column. Consider the delivery_fee values in each row already included the effect of loyalty discount, it would not be a good set of data to be used as a training dataset if the objective is to obtain delivery fee value based on the forementioned three factors.

2. Outlier dataframe:<br>
Given that outliers exist and only exist in the delivery_fee column, it would not be a good training dataset for the model.

3. Missing dataframe: <br>
As inspected before, the missing file has 100 missing values in the delivery_fee column. However, consider the rest of the 400 rows must contain correct values, the 400 rows of the missing dataframe could be a good set of training data to study the relationship between the three factors and delivery fee.

Missing dataframe would be taken as the training data

In [ ]:
# Create subset of missing dataset, where missing values are filtered out
missing_sub = missing[missing["delivery_fee"].notnull()]

# Such subset should contain only 400 rows of data
missing_sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 0 to 499
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   order_id                 400 non-null    object 
 1   date                     400 non-null    object 
 2   time                     400 non-null    object 
 3   order_type               400 non-null    object 
 4   branch_code              400 non-null    object 
 5   order_items              400 non-null    object 
 6   order_price              400 non-null    float64
 7   customer_lat             400 non-null    float64
 8   customer_lon             400 non-null    float64
 9   customerHasloyalty?      400 non-null    int64  
 10  distance_to_customer_KM  400 non-null    float64
 11  delivery_fee             400 non-null    float64
dtypes: float64(5), int64(1), object(6)
memory usage: 40.6+ KB


In [ ]:
# Create empty lists to contain categorical values to determine if a date is on weekend, and to determine the time of the day of an order. 
weekend_or_not = []
time_of_day = []

# Access each date in the dataframe with the function in datetime package,
# to test if the date is weekend (value = 1) or weekday (value = 0)
for idx, row in missing_sub.iterrows():
  date = dt.strptime(row["date"], "%Y-%m-%d")
  date_idx = date.weekday()
  # As the value starts at 0 (i.e. Monday = 0), value 5 and 6 represents Saturday and Sunday respectively
  if date_idx < 5:
    weekend_or_not.append("0")
  else:
    weekend_or_not.append("1")
  # Create a second IF loop to classify the time of the day for orders
  # 0: breakfast, 1: lunch, 2: dinner
  if row["order_type"] == "Breakfast":
    time_of_day.append("0")
  elif row["order_type"] == "Lunch":
    time_of_day.append("1")
  else:
    time_of_day.append("2")

# Adding the two lists created as extra columns for the subset
missing_sub["weekend_or_not"] = weekend_or_not
missing_sub["time_of_day"] = time_of_day

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


As branches apply different method to compute order's delivery fee (but based on the same three factors), three different models should built, addressing for orders for different branch.

Therefore, the subset should be further divided, grouping by the branch_code responsible for an order.

Three sub_subsets are created, which each contains only the orders for a single restaurant branch.

In [ ]:
# Three sub_subset create
missing_sub_NS = missing_sub[missing_sub['branch_code'] == "NS"]
missing_sub_TP = missing_sub[missing_sub['branch_code'] == "TP"]
missing_sub_BK = missing_sub[missing_sub['branch_code'] == "BK"]

In [ ]:
# Establish models on the subsets, each branch should have their own computation method
# While each model contains the same factors: distance, if it was on weekend, and the time of the day for the order

# NS branch
impute_NS = LinearRegression()
impute_NS.fit(missing_sub_NS[["distance_to_customer_KM", "weekend_or_not", "time_of_day"]], missing_sub_NS["delivery_fee"])

# TP branch
impute_TP = LinearRegression()
impute_TP.fit(missing_sub_TP[["distance_to_customer_KM", "weekend_or_not", "time_of_day"]], missing_sub_TP["delivery_fee"])

# BK branch
impute_BK = LinearRegression()
impute_BK.fit(missing_sub_BK[["distance_to_customer_KM", "weekend_or_not", "time_of_day"]], missing_sub_BK["delivery_fee"])

LinearRegression()

So, after establishing models for delivery fee for each branch, it could now be applied for the dirty dataframe.

In [ ]:
# Create a copy of the dirty dataframe to prevent altering the format of the dirty dataframe
dirty_copy = dirty.copy()

In [ ]:
# Apply the same method, adding columsn to indicate if an order is on weekend, and the time of the day of the order

# Create dummy lists
weekend_or_not = []
time_of_day = []

# Access each date in the dataframe with the function in datetime package,
# to test if the date is weekend (value = 1) or weekday (value = 0
for idx, row in dirty_copy.iterrows():
  date = dt.strptime(row["date"], "%Y-%m-%d")
  date_idx = date.weekday()
  # As the value starts at 0 (i.e. Monday = 0), value 5 and 6 represents Saturday and Sunday respectively
  if date_idx < 5:
    weekend_or_not.append("0")
  else:
    weekend_or_not.append("1")
  # Create a second IF loop to classify the time of the day for orders
  # 0: breakfast, 1: lunch, 2: dinner
  if row["order_type"] == "Breakfast":
    time_of_day.append("0")
  elif row["order_type"] == "Lunch":
    time_of_day.append("1")
  else:
    time_of_day.append("2")
    
dirty_copy["weekend_or_not"] = weekend_or_not
dirty_copy["time_of_day"] = time_of_day

In [ ]:
# Grouping orders by their branch code before applying the models, due to the different methods to compute for delivery fee for different branch
NS = dirty_copy[dirty_copy["branch_code"] == "NS"]
TP = dirty_copy[dirty_copy["branch_code"] == "TP"]
BK = dirty_copy[dirty_copy["branch_code"] == "BK"]

# Apply models on each of the subsets to predict the delivery fee
NS["prediction_fee"] = impute_NS.predict(NS[["distance_to_customer_KM", "weekend_or_not", "time_of_day"]])
TP["prediction_fee"] = impute_TP.predict(TP[["distance_to_customer_KM", "weekend_or_not", "time_of_day"]])
BK["prediction_fee"] = impute_BK.predict(BK[["distance_to_customer_KM", "weekend_or_not", "time_of_day"]])

# Combining the three subsets, and merging them back to dirty_copy dataframe, base on order_id, which is the unique identifier of an order
df = pd.concat([NS, TP, BK])
dirty_copy = dirty_copy.merge(df[["order_id", "prediction_fee"]], how = "left", on = "order_id")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

Now that the prediction of delivery fee is done. Decision rule has to be set up to verify the loyalty column in the dirty dataset.

There are four possibilities:
1. Delivery fee is approxmiatley equal to 0.5 * computed value, while customer was labelled with loyalty(True positive)

2. Delivery fee is approxmiatley equal to 0.5 * computed value, while customer was labelled with no loyalty(False negative)

3. Delivery fee is approxmiatley equal to computed value, while customer was labelled with loyalty(False positive)

4. Delivery fee is approxmiatley equal to computed value, while customer was labelled with no loyalty(True negative)

Scenario 2 and scenario 3 are the two types need to be identified.

For scenario 2, if the percentage change of the delivery_fee is close to 50% when compared to the computed value, while customer is labelled without loyalty, it should be corrected

As for scenario 3, if the percentage change of the delivery fee is close to the computed value, while customer is labeld with loyalty, it should be correct as well.

To define "close", a 10% error should be allowed for the model due to the limited size of training dataset.

Based on decision rules stated, first we inspect the rows that are suspected to contain error.

In [ ]:
# Create empty list to contain index of rows with error value
error_loyalty = []

for idx, row in dirty_copy.iterrows():
  fee = row["delivery_fee"]
  prediction = row["prediction_fee"]
  # apply the absolute value function, as we are only concerned with the magnitude of percentage change
  if abs((fee / prediction) - 1) >= 0.4 and row["customerHasloyalty?"] == 0:
    error_loyalty.append(idx)
  elif abs((fee/ prediction) - 1) <= 0.1 and row["customerHasloyalty?"] == 1:
    error_loyalty.append(idx)

In [ ]:
# Inspect rows with error value
dirty_copy.loc[error_loyalty, ]

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee,weekend_or_not,time_of_day,prediction_fee
149,ORDX07596,2021-04-13,11:22:49,Breakfast,BK,"[('Coffee', 5), ('Eggs', 4), ('Pancake', 2), (...",196.0,-37.820241,144.985496,1,6.309,10.753681,0,0,10.322920
159,ORDA05254,2021-09-29,16:57:27,Dinner,BK,"[('Pasta', 7), ('Fish&Chips', 1), ('Shrimp', 10)]",767.5,-37.807807,144.961403,1,7.593,14.426213,0,2,14.765398
252,ORDY00427,2021-05-29,12:54:05,Lunch,TP,"[('Steak', 6), ('Burger', 8), ('Salad', 10), (...",942.0,-37.819949,144.973440,1,7.751,12.390214,1,1,12.900075
261,ORDJ01863,2021-06-27,13:54:55,Lunch,TP,"[('Burger', 4), ('Steak', 8), ('Fries', 4)]",532.0,-37.805456,144.997162,1,11.222,15.356452,1,1,15.404337
283,ORDA03919,2021-09-01,16:06:45,Dinner,BK,"[('Shrimp', 10), ('Fish&Chips', 8), ('Pasta', ...",1354.0,-37.807713,144.973242,1,6.530,13.675309,0,2,13.738230
338,ORDZ10466,2021-08-31,15:56:37,Lunch,NS,"[('Steak', 6), ('Burger', 7), ('Chicken', 7)]",711.0,-37.813478,144.965662,1,7.839,13.487161,0,1,12.714695
447,ORDY10384,2021-08-21,15:36:20,Lunch,TP,"[('Chicken', 3), ('Burger', 2), ('Steak', 5), ...",407.0,-37.816167,144.963974,1,7.788,12.615541,1,1,12.926770


As inspected, the result shows that only scenario 3 (False positive) occurs.

One thing these rows have in common is that their delivery_fee (error-free, always correct) is very close to the computed value (delivery fee generated by models, based on the three factors stated before and it does not include customer loyalty).

In such case, it is believed that the loyalty column for these rows are incorrect and they should be revised.


In [ ]:
# Revise rows with error
for idx, row in dirty_copy.iterrows():
  fee = row["delivery_fee"]
  prediction = row["prediction_fee"]
  # apply the absolute value function, as we are only concerned with the magnitude of percentage change
  if abs((fee / prediction) - 1) >= 0.4 and row["customerHasloyalty?"] == 0:
    dirty_copy.loc[idx, "customerHasloyalty?"] = 1
  elif abs((fee/ prediction) - 1) <= 0.1 and row["customerHasloyalty?"] == 1:
    dirty_copy.loc[idx, "customerHasloyalty?"] = 0

In [ ]:
# Now that all rows with error value in loyalty column are now corrected in dirty_copy dataframe
dirty_copy.loc[error_loyalty, ]

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee,weekend_or_not,time_of_day,prediction_fee
149,ORDX07596,2021-04-13,11:22:49,Breakfast,BK,"[('Coffee', 5), ('Eggs', 4), ('Pancake', 2), (...",196.0,-37.820241,144.985496,0,6.309,10.753681,0,0,10.322920
159,ORDA05254,2021-09-29,16:57:27,Dinner,BK,"[('Pasta', 7), ('Fish&Chips', 1), ('Shrimp', 10)]",767.5,-37.807807,144.961403,0,7.593,14.426213,0,2,14.765398
252,ORDY00427,2021-05-29,12:54:05,Lunch,TP,"[('Steak', 6), ('Burger', 8), ('Salad', 10), (...",942.0,-37.819949,144.973440,0,7.751,12.390214,1,1,12.900075
261,ORDJ01863,2021-06-27,13:54:55,Lunch,TP,"[('Burger', 4), ('Steak', 8), ('Fries', 4)]",532.0,-37.805456,144.997162,0,11.222,15.356452,1,1,15.404337
283,ORDA03919,2021-09-01,16:06:45,Dinner,BK,"[('Shrimp', 10), ('Fish&Chips', 8), ('Pasta', ...",1354.0,-37.807713,144.973242,0,6.530,13.675309,0,2,13.738230
338,ORDZ10466,2021-08-31,15:56:37,Lunch,NS,"[('Steak', 6), ('Burger', 7), ('Chicken', 7)]",711.0,-37.813478,144.965662,0,7.839,13.487161,0,1,12.714695
447,ORDY10384,2021-08-21,15:36:20,Lunch,TP,"[('Chicken', 3), ('Burger', 2), ('Steak', 5), ...",407.0,-37.816167,144.963974,0,7.788,12.615541,1,1,12.926770


In [ ]:
# As dirty and dirty_copy share the same index order, dirty could directly adapt dirty_copy column for customer loyalty value
dirty["customerHasloyalty?"] = dirty_copy["customerHasloyalty?"]

And now the opeartion to fix dirty_data has been completed.

-------------------------------------

<div class="alert alert-block alert-warning">

## 5. Impute the missing values in missing_data.csv <a class="anchor" name="missing"></a>
    
</div>

For missing data, it can be seen with the info() function that missing values exist in delivery_fee (100 rows) and branch_code (50 rows) column.

In [ ]:
missing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   order_id                 500 non-null    object 
 1   date                     500 non-null    object 
 2   time                     500 non-null    object 
 3   order_type               500 non-null    object 
 4   branch_code              450 non-null    object 
 5   order_items              500 non-null    object 
 6   order_price              500 non-null    float64
 7   customer_lat             500 non-null    float64
 8   customer_lon             500 non-null    float64
 9   customerHasloyalty?      500 non-null    int64  
 10  distance_to_customer_KM  500 non-null    float64
 11  delivery_fee             400 non-null    float64
dtypes: float64(5), int64(1), object(6)
memory usage: 47.0+ KB


-------------------------------------

<div class="alert alert-block alert-warning">
    
## 5.1 imputing branch_code <a class="anchor" name="br_missing"></a>
 </div>

Given that missing.csv contains no error value, branch codes for each order could be imputed by comparing the distance between each branch to customer destination. 

Method to computed for distance from session 4.8 could be referenced and applied here.

In [ ]:
branches

,branch_code,node,lat,lon
0,NS,2455254505,-37.773803,144.983647
1,TP,1390575046,-37.861835,144.905716
2,BK,1889485053,-37.815834,145.046450


In [ ]:
nodes

,node,lat,lon
0,711327755,-37.807675,144.955873
1,711327756,-37.807094,144.955978
2,711327759,-37.807301,144.957817
3,711327760,-37.807885,144.957719
4,777781264,-37.805221,144.952173
...,...,...,...
17112,767688602,-37.804021,144.969444
17113,777781181,-37.805238,144.952321
17114,1449431614,-37.815345,144.974603
17115,1449431622,-37.815262,144.975149


In [ ]:
# Getting subset of missing.csv where branch_code value is null
missing_sub = missing[missing["branch_code"].isnull()]

# Create a merged set which includes order information as well as the destination node id
dest = missing_sub[["order_id", "customer_lat", "customer_lon", "distance_to_customer_KM"]]
dest = dest.merge(nodes, how = "left", left_on = ["customer_lat", "customer_lon"], right_on = ["lat", "lon"])
dest = dest[["order_id", "distance_to_customer_KM", "node", "lat", "lon"]]
dest

,order_id,distance_to_customer_KM,node,lat,lon
0,ORDX05191,4.192,443297188,-37.816484,145.005003
1,ORDJ02302,9.755,579489433,-37.815958,144.950693
2,ORDC05684,8.881,660546021,-37.812003,144.990883
3,ORDI01877,9.538,266739860,-37.823357,144.989107
4,ORDI02376,8.126,3132496573,-37.815015,144.964298
5,ORDX00453,5.409,4315295089,-37.815822,144.992783
6,ORDX02429,10.935,1201417396,-37.799545,144.927418
7,ORDZ00344,10.055,3810637328,-37.820999,144.943166
8,ORDA00363,8.149,662736521,-37.824599,144.969813
9,ORDJ00888,7.753,197020302,-37.821450,144.969046


In [ ]:
# From the branches dataframe, obtain node id for branches
NS = branches[branches["branch_code"] == "NS"].iloc[0,1]
TP = branches[branches["branch_code"] == "TP"].iloc[0,1]
BK = branches[branches["branch_code"] == "BK"].iloc[0,1]

# Create empty list to contain branch_code value
branch_list = []
for idx, row in dest.iterrows():
  # Get customer location represented in node
  destination = row["node"]
  # Compute distance between customer location and branches
  # Round up values to 3 decimal places to follow the dataframe format
  NS_dist = round(nx.shortest_path_length(G, source = NS, target = destination, weight = "weight"), 3)
  TP_dist = round(nx.shortest_path_length(G, source = TP, target = destination, weight = "weight"), 3)
  BK_dist = round(nx.shortest_path_length(G, source = BK, target = destination, weight = "weight"), 3)
  # Compare each branch distance to destination.
  # If one is matched, it can be determined that the order was delivered from the matched branch
  if row["distance_to_customer_KM"] == NS_dist:
    branch_list.append("NS")
  elif row["distance_to_customer_KM"] == TP_dist:
    branch_list.append("TP")
  else:
    branch_list.append("BK")

In [ ]:
# adding branch_code value to the subset of missing.csv
missing_sub["branch_code"] = branch_list

# Hence, missing branch_code value could be restored in missing dataframe by index, in order to preserve the oringial format of the file. 
for idx, row in missing_sub.iterrows():
  missing.loc[idx, "branch_code"] = row["branch_code"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# Missing values in missing dataframe restored
missing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   order_id                 500 non-null    object 
 1   date                     500 non-null    object 
 2   time                     500 non-null    object 
 3   order_type               500 non-null    object 
 4   branch_code              500 non-null    object 
 5   order_items              500 non-null    object 
 6   order_price              500 non-null    float64
 7   customer_lat             500 non-null    float64
 8   customer_lon             500 non-null    float64
 9   customerHasloyalty?      500 non-null    int64  
 10  distance_to_customer_KM  500 non-null    float64
 11  delivery_fee             400 non-null    float64
dtypes: float64(5), int64(1), object(6)
memory usage: 47.0+ KB


-------------------------------------

<div class="alert alert-block alert-warning">
    
## 5.2 imputing delivery fee <a class="anchor" name="fee_missing"></a>
 </div>

As missing branched_code values are now restored, by referring to the ordering branch for each order in missing dataframe, regression models built in session 4.9 could now be applied. 

In such way, delivery fee could be approximated for rows that contain missing value in delivery_fee column.

In [ ]:
# Branches adopt different methods to calculate delivery fee, all three models are required.
# Also the missing value need to be grouped by their branch_code

# Subset of Missing dataframe where delivery_fee value is missing
missing_sub = missing[missing["delivery_fee"].isnull()]


In [ ]:
missing_sub

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee
8,ORDX05191,2021-03-29,10:52:23,Breakfast,BK,"[('Pancake', 1), ('Coffee', 6), ('Cereal', 2),...",137.25,-37.816484,145.005003,0,4.192,NaN
9,ORDI05894,2021-10-03,17:07:36,Dinner,NS,"[('Fish&Chips', 1), ('Shrimp', 7), ('Salmon', ...",673.50,-37.806155,144.945897,0,8.960,NaN
13,ORDI07793,2021-04-16,11:22:49,Breakfast,NS,"[('Eggs', 4), ('Coffee', 7), ('Pancake', 8), (...",480.50,-37.822921,144.987480,0,9.478,NaN
27,ORDJ02302,2021-04-23,11:43:05,Breakfast,TP,"[('Eggs', 8), ('Cereal', 6)]",288.00,-37.815958,144.950693,0,9.755,NaN
35,ORDI03932,2021-04-21,11:32:57,Breakfast,NS,"[('Eggs', 10), ('Coffee', 8), ('Pancake', 9), ...",634.25,-37.814161,144.991777,0,9.359,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
463,ORDA03816,2021-06-25,13:44:47,Lunch,BK,"[('Chicken', 6), ('Steak', 8)]",552.00,-37.803762,144.956376,0,8.117,NaN
464,ORDZ10482,2021-09-25,16:47:19,Dinner,NS,"[('Salmon', 7), ('Shrimp', 10)]",827.00,-37.801216,144.962985,0,6.959,NaN
473,ORDZ07083,2021-09-24,16:47:19,Dinner,NS,"[('Fish&Chips', 9), ('Pasta', 6), ('Salmon', 7...",1037.00,-37.811274,144.951838,0,8.582,NaN
479,ORDB08617,2021-02-02,09:00:50,Breakfast,TP,"[('Pancake', 5), ('Eggs', 2)]",153.25,-37.820864,144.920634,0,11.287,NaN


In [ ]:
# Apply the same method as session 4.9, computing for factors that contribute to the regression model
# Create empty lists to contain categorical values to determine if a date is on weekend, and to determine the time of the day of an order. 
weekend_or_not = []
time_of_day = []

# Access each date in the dataframe with the function in datetime package,
# to test if the date is weekend (value = 1) or weekday (value = 0)
for idx, row in missing_sub.iterrows():
  date = dt.strptime(row["date"], "%Y-%m-%d")
  date_idx = date.weekday()
  # As the value starts at 0 (i.e. Monday = 0), value 5 and 6 represents Saturday and Sunday respectively
  if date_idx < 5:
    weekend_or_not.append("0")
  else:
    weekend_or_not.append("1")
  # Create a second IF loop to classify the time of the day for orders
  # 0: breakfast, 1: lunch, 2: dinner
  if row["order_type"] == "Breakfast":
    time_of_day.append("0")
  elif row["order_type"] == "Lunch":
    time_of_day.append("1")
  else:
    time_of_day.append("2")

# Adding the two lists created as extra columns for the subset
missing_sub["weekend_or_not"] = weekend_or_not
missing_sub["time_of_day"] = time_of_day

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
# Separate subset by branch code
BK_missing = missing_sub[missing_sub["branch_code"] == "BK"]
NS_missing = missing_sub[missing_sub["branch_code"] == "NS"]
TP_missing = missing_sub[missing_sub["branch_code"] == "TP"]

# Apply model to subset respectively
NS_missing["delivery_fee"] = impute_NS.predict(NS_missing[["distance_to_customer_KM", "weekend_or_not", "time_of_day"]])
TP_missing["delivery_fee"] = impute_TP.predict(TP_missing[["distance_to_customer_KM", "weekend_or_not", "time_of_day"]])
BK_missing["delivery_fee"] = impute_BK.predict(BK_missing[["distance_to_customer_KM", "weekend_or_not", "time_of_day"]])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [ ]:
# Combining the three subsets, and merging them back to dirty_copy dataframe, base on order_id, which is the unique identifier of an order
df = pd.concat([NS_missing, TP_missing, BK_missing])
missing_sub = missing_sub[["order_id", "customerHasloyalty?"]].merge(df[["order_id", "delivery_fee"]], how = "left", on = "order_id").set_axis(missing_sub.index)

In [ ]:
# Notice that the models do not consider the effect of loyalty discount on delivery cost.
# Therefore, this need to be handled separately for customer with loyalty
for idx, row in missing_sub.iterrows():
  if row["customerHasloyalty?"] == 1:
    missing_sub.loc[idx, "delivery_fee"] = row["delivery_fee"] / 2  # Replace value with 50% discount for customer with loyalty

In [ ]:
# Finally, restore delivery fee computed to the Missing dataframe by index
for idx, row in missing_sub.iterrows():
  missing.loc[idx, "delivery_fee"] = row["delivery_fee"]

In [ ]:
# And all missing values in Missing dataframe are now completely imputed.
missing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   order_id                 500 non-null    object 
 1   date                     500 non-null    object 
 2   time                     500 non-null    object 
 3   order_type               500 non-null    object 
 4   branch_code              500 non-null    object 
 5   order_items              500 non-null    object 
 6   order_price              500 non-null    float64
 7   customer_lat             500 non-null    float64
 8   customer_lon             500 non-null    float64
 9   customerHasloyalty?      500 non-null    int64  
 10  distance_to_customer_KM  500 non-null    float64
 11  delivery_fee             500 non-null    float64
dtypes: float64(5), int64(1), object(6)
memory usage: 47.0+ KB


-------------------------------------

<div class="alert alert-block alert-warning">

## 6. Detect and remove outlier rows <a class="anchor" name="outliers"></a>
    
</div>

Lastly, anomolies within Outlier dataframe will be handled.

It was instructed that outlier.csv file contains no errors value, while outliers only exist with respect to the delivery_fee column.

In such case, each row represents a real-world observation, which could not be directly neglected and removed.

Therefore, it is essential to adopt linear regression models to Outlier dataset, in order to understand the possible issue with the exisint values in the delivery_fee column.

By fixing Dirty dataframe and Missing dataframe, it is believed that both datasets are now cleaned, and could be applied as training data. Such that with a larger training dataset, it is expected to see more accurate prediction result when applied to the Outlier dataset.

In [ ]:
outlier.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   order_id                 500 non-null    object 
 1   date                     500 non-null    object 
 2   time                     500 non-null    object 
 3   order_type               500 non-null    object 
 4   branch_code              500 non-null    object 
 5   order_items              500 non-null    object 
 6   order_price              500 non-null    float64
 7   customer_lat             500 non-null    float64
 8   customer_lon             500 non-null    float64
 9   customerHasloyalty?      500 non-null    int64  
 10  distance_to_customer_KM  500 non-null    float64
 11  delivery_fee             500 non-null    float64
dtypes: float64(5), int64(1), object(6)
memory usage: 47.0+ KB


In [ ]:
# Combining Dirty and Missing dataframe
combined_df = dirty.append(missing, ignore_index = True)

# Info of the combined dataframe
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   order_id                 1000 non-null   object 
 1   date                     1000 non-null   object 
 2   time                     1000 non-null   object 
 3   order_type               1000 non-null   object 
 4   branch_code              1000 non-null   object 
 5   order_items              1000 non-null   object 
 6   order_price              1000 non-null   float64
 7   customer_lat             1000 non-null   float64
 8   customer_lon             1000 non-null   float64
 9   customerHasloyalty?      1000 non-null   int64  
 10  distance_to_customer_KM  1000 non-null   float64
 11  delivery_fee             1000 non-null   float64
dtypes: float64(5), int64(1), object(6)
memory usage: 93.9+ KB


In [ ]:
# Applying the same method to set up models

# Create empty lists to contain categorical values to determine if a date is on weekend, and to determine the time of the day of an order. 
weekend_or_not = []
time_of_day = []

# Access each date in the dataframe with the function in datetime package,
# to test if the date is weekend (value = 1) or weekday (value = 0)
for idx, row in combined_df.iterrows():
  date = dt.strptime(row["date"], "%Y-%m-%d")
  date_idx = date.weekday()
  # As the value starts at 0 (i.e. Monday = 0), value 5 and 6 represents Saturday and Sunday respectively
  if date_idx < 5:
    weekend_or_not.append("0")
  else:
    weekend_or_not.append("1")
  # Create a second IF loop to classify the time of the day for orders
  # 0: breakfast, 1: lunch, 2: dinner
  if row["order_type"] == "Breakfast":
    time_of_day.append("0")
  elif row["order_type"] == "Lunch":
    time_of_day.append("1")
  else:
    time_of_day.append("2")

# Adding the two lists created as extra columns for the subset
combined_df["weekend_or_not"] = weekend_or_not
combined_df["time_of_day"] = time_of_day

# Three sub_subset create
combined_NS = combined_df[combined_df['branch_code'] == "NS"]
combined_TP = combined_df[combined_df['branch_code'] == "TP"]
combined_BK = combined_df[combined_df['branch_code'] == "BK"]


# Establish models on the subsets, each branch should have their own computation method
# While each model contains the same factors: distance, if it was on weekend, and the time of the day for the order

# NS branch
impute_NS = LinearRegression()
impute_NS.fit(combined_NS[["distance_to_customer_KM", "weekend_or_not", "time_of_day"]], combined_NS["delivery_fee"])

# TP branch
impute_TP = LinearRegression()
impute_TP.fit(combined_TP[["distance_to_customer_KM", "weekend_or_not", "time_of_day"]], combined_TP["delivery_fee"])

# BK branch
impute_BK = LinearRegression()
impute_BK.fit(combined_BK[["distance_to_customer_KM", "weekend_or_not", "time_of_day"]], combined_BK["delivery_fee"])

LinearRegression()

In [ ]:
# Create a copy of Outlier dataframe for further operations
# In order to preserve the format and arrangement of the original dataset
outlier_copy = outlier.copy()

In [ ]:
# Obtain features for model building by applying the same method to Outlier dataset
# Create empty lists to contain categorical values to determine if a date is on weekend, and to determine the time of the day of an order. 
weekend_or_not = []
time_of_day = []

# Access each date in the dataframe with the function in datetime package,
# to test if the date is weekend (value = 1) or weekday (value = 0)
for idx, row in outlier_copy.iterrows():
  date = dt.strptime(row["date"], "%Y-%m-%d")
  date_idx = date.weekday()
  # As the value starts at 0 (i.e. Monday = 0), value 5 and 6 represents Saturday and Sunday respectively
  if date_idx < 5:
    weekend_or_not.append("0")
  else:
    weekend_or_not.append("1")
  # Create a second IF loop to classify the time of the day for orders
  # 0: breakfast, 1: lunch, 2: dinner
  if row["order_type"] == "Breakfast":
    time_of_day.append("0")
  elif row["order_type"] == "Lunch":
    time_of_day.append("1")
  else:
    time_of_day.append("2")

# Adding the two lists created as extra columns for the subset
outlier_copy["weekend_or_not"] = weekend_or_not
outlier_copy["time_of_day"] = time_of_day

In [ ]:
# Create subsets for Outlier dataset, grouping by branch_code
NS = outlier_copy[outlier_copy["branch_code"] == "NS"]
TP = outlier_copy[outlier_copy["branch_code"] == "TP"]
BK = outlier_copy[outlier_copy["branch_code"] == "BK"]

# Apply models on each of the subsets to predict the delivery fee
NS["prediction_fee"] = impute_NS.predict(NS[["distance_to_customer_KM", "weekend_or_not", "time_of_day"]])
TP["prediction_fee"] = impute_TP.predict(TP[["distance_to_customer_KM", "weekend_or_not", "time_of_day"]])
BK["prediction_fee"] = impute_BK.predict(BK[["distance_to_customer_KM", "weekend_or_not", "time_of_day"]])

# Combining the three subsets, and merging them back to dirty_copy dataframe, base on order_id, which is the unique identifier of an order
df = pd.concat([NS, TP, BK])
outlier_copy = outlier_copy.merge(df[["order_id", "prediction_fee"]], how = "left", on = "order_id")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [ ]:
# To obtain accurate value to compare with the original delivery_fee value, also include the effect of customer loyalty discount
for idx, row in outlier_copy.iterrows():
  if row["customerHasloyalty?"] == 1:
    outlier_copy.loc[idx, "prediction_fee"] = row["prediction_fee"] * 0.5

In [ ]:
outlier_copy.describe()

,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee,prediction_fee
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,472.316200,-37.812487,144.966685,0.038000,8.733456,13.968614,13.591216
std,258.568117,0.007753,0.021187,0.191388,1.578963,3.447212,2.547858
min,31.000000,-37.829895,144.917550,0.000000,3.715000,5.006290,4.803066
25%,276.150000,-37.818861,144.951979,0.000000,7.822250,12.628014,12.389034
50%,428.000000,-37.812701,144.963530,0.000000,8.751500,13.990794,13.649937
75%,624.625000,-37.806093,144.982457,0.000000,9.747000,15.348281,15.026761
max,1250.000000,-37.788265,145.017015,1.000000,13.954000,31.088812,21.183334


In [ ]:
# Create new column containing the difference between the delivery_fee value to the predicted value.
outlier_copy["computed_diff"] = outlier_copy["delivery_fee"] - outlier_copy["prediction_fee"]

As stated previously, the linear regression models were built upon three features:
1. Distance
2. Weekend or weekday
3. Time or the day

Based on these three features, a line of best fit was produced. If there is an error, which a model with only these three features could not explained, it is considered as a outlier.

When such outlier exists, it should be fixed if possible, or in this case when no further information was given, then it should be deleted instead. Or if the data is correct, it should be keep within the dataset.

No fixed rule is given to determine outliers. In this scenario, any datapoints within two standard deviation away from the regression line (horizontal distance) will be forgiven. Otherwise, if a data point is more than two stadard deviation away, it will be removed.


First, compute standard deviation with the following formula:

standard deviation = square_root( SSE / (n - 2)), 

where sse is the sum of squared error, and n is the size of data

In [ ]:
# First, get SSE
mean = outlier_copy["computed_diff"].mean()

# Create emtpty list to obtain errors
Sum_error = []
for idx, row in outlier_copy.iterrows():
  Sum_error.append((row["computed_diff"] - mean) ** 2)

# Adding up all sum_error values as SSE
sse = sum(Sum_error)

# Count data points
n = len(outlier_copy.index)

# Obtain standard deviation
sd = math.sqrt(sse / (n - 2))


In [ ]:
# obtain indexes of rows which contain outlier
outlier_idx = outlier_copy[outlier_copy["computed_diff"] > sd * 2].index


The method applied is called residual analysis. It is commonly applied to detect outliers which are not neccessarily extreme values, but with large residual that might affect the line of best fit significantly. 

By removing these rows with large residuals, it is expected to see a better fitted regression line for the model.

In [ ]:
# Display outliers from residual analysis
outlier_copy.loc[outlier_idx,][["delivery_fee", "prediction_fee", "computed_diff"]]

,delivery_fee,prediction_fee,computed_diff
8,21.402680,13.703397,7.699282
9,19.184124,13.263650,5.920474
13,19.806723,12.288557,7.518167
26,20.921877,13.683897,7.237979
51,31.088812,20.723724,10.365088
70,23.441096,14.830782,8.610314
109,18.173494,11.981517,6.191977
131,19.349668,12.378936,6.970732
147,21.573253,14.550232,7.023021
162,22.548089,14.726562,7.821527


In [ ]:
# Removing outliers from outliers
outlier.drop(outlier_idx, axis = 0, inplace = True)

In [ ]:
outlier_copy

,order_id,date,time,order_type,branch_code,order_items,order_price,customer_lat,customer_lon,customerHasloyalty?,distance_to_customer_KM,delivery_fee,weekend_or_not,time_of_day,prediction_fee,computed_diff
0,ORDK06792,2021-01-01,08:00:00,Breakfast,BK,"[('Pancake', 3), ('Coffee', 1), ('Cereal', 1)]",92.25,-37.811589,144.940783,1,9.637,6.752846,0,0,6.692070,0.060775
1,ORDK02204,2021-06-11,13:14:21,Lunch,BK,"[('Steak', 6), ('Fries', 10), ('Chicken', 3), ...",844.00,-37.825079,144.982778,0,7.106,13.282917,0,1,12.675384,0.607533
2,ORDJ09116,2021-09-22,16:47:19,Dinner,TP,"[('Shrimp', 9), ('Pasta', 2), ('Fish&Chips', 10)]",891.00,-37.816570,144.990058,0,9.790,13.821452,0,2,13.997285,-0.175833
3,ORDY10134,2021-05-19,12:33:48,Lunch,TP,"[('Steak', 1), ('Burger', 10)]",355.00,-37.813224,144.948536,0,10.216,13.226014,0,1,12.999300,0.226714
4,ORDJ04394,2021-11-29,18:59:09,Dinner,TP,"[('Fish&Chips', 7), ('Shrimp', 9)]",731.00,-37.824513,144.982190,0,8.873,6.503919,0,2,13.305946,-6.802027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,ORDC07319,2021-06-04,13:04:13,Lunch,NS,"[('Salad', 2), ('Fries', 9), ('Steak', 10)]",592.40,-37.815505,144.935488,0,10.059,15.576341,0,1,15.164942,0.411399
496,ORDC08721,2021-12-06,19:19:26,Dinner,NS,"[('Shrimp', 2), ('Salmon', 7), ('Fish&Chips', 1)]",430.00,-37.813224,144.958623,0,8.149,14.838490,0,2,14.239605,0.598885
497,ORDK06098,2021-08-02,15:05:54,Lunch,BK,"[('Chicken', 6), ('Steak', 7), ('Burger', 8), ...",818.60,-37.807329,144.968732,0,6.913,12.712001,0,1,12.500424,0.211577
498,ORDI02035,2021-04-14,11:22:49,Breakfast,NS,"[('Cereal', 4), ('Coffee', 10), ('Pancake', 2)]",179.50,-37.824281,144.985102,0,9.467,14.278792,0,0,13.421601,0.857191


-------------------------------------

<div class="alert alert-block alert-warning">

## 7. Export csv files <a class="anchor" name="write"></a>
    
</div>

In [ ]:
# # Set up directory
os.chdir("/content/drive/MyDrive/5196A2")

# # Export csv file
dirty.to_csv("32973381_dirty_data_solution.csv", index = False)
missing.to_csv("32973381_missing_data_solution.csv", index = False)
outlier.to_csv("32973381_outlier_data_solution.csv", index = False)

-------------------------------------

<div class="alert alert-block alert-warning"> 

## 8. Summary <a class="anchor" name="summary"></a>

</div>

From session 6 of this assignment, Outlier dataframe was cleansed and 25 rows of outliers were removed. This proves the assumption in session 4.9 that the Missing dataframe was adopted instead of the Outlier dataframe, when the regression models were first developed. Due to the uncertainty in Outliers dataframe that a outlier may exist in any row, which would affect the fitness of a model significantly, it is not the best data to be adopted to when delivery_fee is the value to be computed.


From all the operations, it is understood that the job of a data analyst or a data scientist will be a lot tougher given that their works do not have the restriction of one error per row, or one type of error in a given document, etc.



-------------------------------------

<div class="alert alert-block alert-warning"> 

## 9. References <a class="anchor" name="ref"></a>

</div>



[1]<a class="anchor" name="ref-2"></a> Libre Texts Statistics - Introductory Statistics - Linear_Regression_and_Correlation,https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(OpenStax)/12%3A_Linear_Regression_and_Correlation/12.07%3A_Outliers, Accessed on 5/10/2022

[2]<a class="anchor" name="ref-2"></a> Data Preparation
, Zahraa S. Abdallah, Lan Du, Geoffrey I. Webb, Accessed on 23/09/2022

[3]<a class="anchor" name="ref-2"></a>PennState Eberly Colledge of Science - Identifying Outliers, https://online.stat.psu.edu/stat462/node/172/, Accessed on 23/08/2022


-------------------------------------